In [114]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.impute import KNNImputer

df = pd.read_csv('../train.csv')
df = df.drop(columns=['Name', 'PassengerId'])

binary_features = ['VIP', 'CryoSleep']
cryo_sleep_depending_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
categorical_features_without_cabin = ['HomePlanet', 'Destination']
categorical_features_cleared = ['HomePlanet', 'Destination', 'Deck', 'Side']
numerical_features = ['ShoppingMall', 'Spa', 'VRDeck', 'RoomService', 'FoodCourt', 'Age']
imputer = KNNImputer(n_neighbors=1)

for i in range(len(df)):
    if df.at[i, 'CryoSleep'] == True:
        for feature in cryo_sleep_depending_features:
            df.at[i, feature] = 0.0

rows_with_many_missing_values = df[df.isna().sum(axis=1) > 1]
rows_with_one_missing_value = df[df.isna().sum(axis=1) == 1]
df = df.dropna()     
df[["Deck", "CabinNumber", "Side"]] = df["Cabin"].str.split("/", expand=True)
df = df.drop(columns=['Cabin', 'CabinNumber'])
df = df.astype({'CryoSleep': int, 'VIP': int, 'Transported': int})
df = pd.get_dummies(df, columns=categorical_features_cleared)

for feature in numerical_features:
    rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value[feature].isna()] 
    rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
    rows_with_missing_feature = rows_with_missing_feature.drop(columns=['Cabin', 'CabinNumber'])
    rows_with_missing_feature = rows_with_missing_feature.astype({'CryoSleep': int, 'VIP': int, 'Transported': int})
    rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_cleared)
    df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

for feature in categorical_features_without_cabin:
    rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value[feature].isna()] 
    rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
    rows_with_missing_feature = rows_with_missing_feature.drop(columns=['Cabin', 'CabinNumber'])
    rows_with_missing_feature = rows_with_missing_feature.astype({'CryoSleep': int, 'VIP': int, 'Transported': int})
    rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_cleared)
    df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

for feature in binary_features:
    rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value[feature].isna()] 
    rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
    rows_with_missing_feature = rows_with_missing_feature.drop(columns=['Cabin', 'CabinNumber'])
    rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_cleared)
    df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

rows_with_missing_feature = rows_with_one_missing_value[rows_with_one_missing_value['Cabin'].isna()] 
rows_with_missing_feature = rows_with_missing_feature.drop(columns='Cabin')
rows_with_missing_feature = rows_with_missing_feature.astype({'CryoSleep': int, 'VIP': int, 'Transported': int})
rows_with_missing_feature = pd.get_dummies(rows_with_missing_feature, columns=categorical_features_without_cabin)
df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

rows_with_many_missing_values = rows_with_many_missing_values.drop(columns='Cabin')

df = pd.concat([df, rows_with_missing_feature], ignore_index=True)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

target = 'Transported'
X = df.drop(columns=target)
y = df[target]

model = CatBoostClassifier(random_state=42)

scores = cross_val_score(model, X, y, cv=10)
print("Оценки кросс-валидации:", scores)
print("Средняя оценка:", scores.mean())

C:\Users\vitya\AppData\Local\Temp\ipykernel_14228\1610120963.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
C:\Users\vitya\AppData\Local\Temp\ipykernel_14228\1610120963.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_with_missing_feature[["Deck", "CabinNumber", "Side"]] = rows_with_missing_feature["Cabin"].str.split("/", expand=True)
C:\Users\vitya\AppData\Local\Temp\ip

Learning rate set to 0.024839
0:	learn: 0.6804960	total: 3.17ms	remaining: 3.17s
1:	learn: 0.6669429	total: 5.98ms	remaining: 2.98s
2:	learn: 0.6556814	total: 8.94ms	remaining: 2.97s
3:	learn: 0.6447437	total: 11.8ms	remaining: 2.95s
4:	learn: 0.6347644	total: 14.7ms	remaining: 2.93s
5:	learn: 0.6241282	total: 17.6ms	remaining: 2.91s
6:	learn: 0.6149536	total: 20.4ms	remaining: 2.89s
7:	learn: 0.6063928	total: 23.3ms	remaining: 2.89s
8:	learn: 0.5988829	total: 26.3ms	remaining: 2.89s
9:	learn: 0.5923278	total: 29ms	remaining: 2.87s
10:	learn: 0.5854481	total: 31.7ms	remaining: 2.85s
11:	learn: 0.5792137	total: 34.5ms	remaining: 2.84s
12:	learn: 0.5724759	total: 37.1ms	remaining: 2.82s
13:	learn: 0.5666616	total: 40ms	remaining: 2.82s
14:	learn: 0.5608691	total: 42.9ms	remaining: 2.81s
15:	learn: 0.5556304	total: 45.7ms	remaining: 2.81s
16:	learn: 0.5498077	total: 48.6ms	remaining: 2.81s
17:	learn: 0.5446732	total: 51.3ms	remaining: 2.8s
18:	learn: 0.5401684	total: 54.3ms	remaining: 2.8